In [ ]:
##Pdf Reader
from langchain_community.document_loaders import PyPDFLoader
import os
os.environ["OPENAI_API_KEY"] = "YOUR OPENAI API KEY"
loader=PyPDFLoader("pdf_name.pdf")

docs = loader.load()
docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings() 
db = FAISS.from_documents(documents, embeddings)

In [ ]:
query = "Your query"

result = db.similarity_search(query)
result[0].page_content

In [ ]:
from langchain_openai.llms import OpenAI
llm = OpenAI()
llm 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the context provided.
Think step by step before providing a detailed answer.
<context>
{context}
</context>                                                                                    
Question:{input}""")

In [ ]:
## Create stuff document chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm,prompt)

In [ ]:
retriever = db.as_retriever()
retriever

In [ ]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input":"Your question"})

In [ ]:
response['answer']